In [1]:
import datetime
import sys
import os

import IPython
import IPython.display
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from sklearn.preprocessing import MinMaxScaler

import src.preprocessing_3days
from src.preprocessing_3days import series_to_supervised, preprocess, create_evaluation_df
from src.functions import load_data, TimeSeriesTensor, plot_train_history, validation, save_model, load_model

np.set_printoptions(threshold=sys.maxsize)
pd.options.display.max_seq_items = 2000

In [2]:
def train_test_split(df, n_test):
    if len(df) < 8760:
        n_test = round(len(df) * 0.2)
    test_df = df.copy()[-(n_test+71):]
    train_df = df.copy()[:-(len(test_df)-71)]
    return train_df, test_df

In [3]:
def MIMO_fulldata_preparation(df, n_test=4380, T=72, HORIZON=72):
    df = df.merge(series_to_supervised(df), how='right', left_index=True, right_index=True)
    df = preprocess(df, 'Belgium')
    train_df, test_df = train_test_split(df, n_test)
    y_scaler = MinMaxScaler()
    y_scaler.fit(train_df[['value']])
    long_scaler = MinMaxScaler()
    test_df[test_df.columns] = long_scaler.fit_transform(test_df)
    train_df[train_df.columns] = long_scaler.fit_transform(train_df)
    #print(train_df.columns)
    tensor_structure = {'X':(range(-T+1, 1), train_df.columns[:1]), 'X2':(range(1, 73), train_df.columns[1:6]), 'static':(None, train_df.columns[6:])}
    tensor_structure = {'X':(range(-T+1, 1), train_df.columns[:1]), 'X2':(range(1, 73), train_df.columns[1:])}
    #tensor_structure = {'X':(range(-T+1, 1), train_df.columns)}
    #print(tensor_structure[0])
    train_inputs = TimeSeriesTensor(train_df, 'value', HORIZON, tensor_structure)
    test_inputs = TimeSeriesTensor(test_df, 'value', HORIZON, tensor_structure)
    return train_inputs, test_inputs, y_scaler

In [4]:
GEP1 = pd.read_csv('../data/GEP/Consumption_1H.csv', index_col=0, header=0, names=['value'])
GEP4 = pd.read_csv('../data/GEP/B4_Consumption_1H.csv', index_col=0, header=0, names=['value'])
datasets = [GEP1, GEP4]
names = ['GEP1', 'GEP4']

In [5]:
train_inputs, test_inputs, y_scaler = MIMO_fulldata_preparation(GEP4, n_test=4380, T=72, HORIZON=72)

In [6]:
train_inputs['X'].shape

(11293, 72, 1)

In [7]:
train_inputs['X2'].shape

(11293, 72, 13)

In [8]:
train_inputs['X'][::24].shape

(471, 72, 1)

In [5]:
MAX_EPOCHS = 150
BATCHSIZE = 32
patience = 10
HORIZON = 72


FULL_LSTMIMO = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(32, input_shape=(HORIZON, 14)),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(HORIZON)
])


metrics = pd.DataFrame(columns=['mae','mape', 'rmse', 'B'], index=range(28))
from progressbar import ProgressBar
pbar = ProgressBar()
dX_train = []
dT_train = []
dX_test = []
dX_scaler = []
for i,df in enumerate(datasets):
    train_inputs, test_inputs, y_scaler = MIMO_fulldata_preparation(df, n_test=4380, T=72, HORIZON=72)
    dX_train.append(tf.concat([train_inputs['X'][::24],train_inputs['X2'][::24]], axis=2))
    dT_train.append(train_inputs['target'][::24])
    dX_test.append(test_inputs)
    dX_scaler.append(y_scaler)
global_inputs_X = tf.concat(dX_train, 0)
global_inputs_T = tf.concat(dT_train, 0)
#test_inputs = pd.concat(dn_test, axis=1)

# full data LSTM MIMO compilation and fit
FULL_LSTMIMO.compile(optimizer=tf.optimizers.Adam(), loss='mse', metrics=[tf.metrics.MeanSquaredError()])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=patience, mode='min')
        
history = FULL_LSTMIMO.fit(global_inputs_X, global_inputs_T, batch_size=180, epochs=MAX_EPOCHS,
                      validation_split=0.15,
                      callbacks=[early_stopping], verbose=1)
save_model(FULL_LSTMIMO, 'new_inputs_model')
for i,df in enumerate(datasets):
    concat_input = tf.concat([dX_test[i]['X'][::24],dX_test[i]['X2'][::24]], axis=2)
    FD_predictions = FULL_LSTMIMO.predict(concat_input)
    FD_eval_df = create_evaluation_df(FD_predictions, dX_test[i], HORIZON, dX_scaler[i])
    mae = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAE')
    mape = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'MAPE')
    rmse = validation(FD_eval_df['prediction'], FD_eval_df['actual'], 'RMSE')
    #print('rmse {}'.format(rmse))
    metrics.loc[i] = pd.Series({'mae':mae, 'mape':mape, 'rmse':rmse, 'B': names[i]})
metrics.to_csv('./results/GEP/global/3days/24h_LSTM.csv')

Train on 1808 samples, validate on 320 samples
Epoch 1/150
1808/1808 [==============================] - 7s 4ms/sample - loss: 0.1481 - mean_squared_error: 0.1481 - val_loss: 0.1200 - val_mean_squared_error: 0.1200
Epoch 2/150
1808/1808 [==============================] - 4s 2ms/sample - loss: 0.1044 - mean_squared_error: 0.1044 - val_loss: 0.0847 - val_mean_squared_error: 0.0847
Epoch 3/150
1808/1808 [==============================] - 4s 2ms/sample - loss: 0.0691 - mean_squared_error: 0.0691 - val_loss: 0.0545 - val_mean_squared_error: 0.0545
Epoch 4/150
1808/1808 [==============================] - 5s 3ms/sample - loss: 0.0427 - mean_squared_error: 0.0427 - val_loss: 0.0377 - val_mean_squared_error: 0.0377
Epoch 5/150
1808/1808 [==============================] - 5s 3ms/sample - loss: 0.0292 - mean_squared_error: 0.0292 - val_loss: 0.0300 - val_mean_squared_error: 0.0300
Epoch 6/150
1808/1808 [==============================] - 6s 3ms/sample - loss: 0.0233 - mean_squared_error: 0.0233 - v

1808/1808 [==============================] - 4s 2ms/sample - loss: 0.0158 - mean_squared_error: 0.0158 - val_loss: 0.0219 - val_mean_squared_error: 0.0219
Epoch 50/150
1808/1808 [==============================] - 4s 2ms/sample - loss: 0.0158 - mean_squared_error: 0.0158 - val_loss: 0.0218 - val_mean_squared_error: 0.0218
Epoch 51/150
1808/1808 [==============================] - 4s 2ms/sample - loss: 0.0158 - mean_squared_error: 0.0158 - val_loss: 0.0220 - val_mean_squared_error: 0.0220
Epoch 52/150
1808/1808 [==============================] - 4s 2ms/sample - loss: 0.0158 - mean_squared_error: 0.0158 - val_loss: 0.0218 - val_mean_squared_error: 0.0218
Epoch 53/150
1808/1808 [==============================] - 4s 2ms/sample - loss: 0.0159 - mean_squared_error: 0.0159 - val_loss: 0.0219 - val_mean_squared_error: 0.0219
Epoch 54/150
1808/1808 [==============================] - 4s 2ms/sample - loss: 0.0157 - mean_squared_error: 0.0157 - val_loss: 0.0217 - val_mean_squared_error: 0.0217
Epoch

Epoch 98/150
1808/1808 [==============================] - 4s 2ms/sample - loss: 0.0152 - mean_squared_error: 0.0152 - val_loss: 0.0215 - val_mean_squared_error: 0.0215
Epoch 99/150
1808/1808 [==============================] - 4s 2ms/sample - loss: 0.0152 - mean_squared_error: 0.0152 - val_loss: 0.0215 - val_mean_squared_error: 0.0215
Epoch 100/150
1808/1808 [==============================] - 4s 2ms/sample - loss: 0.0152 - mean_squared_error: 0.0152 - val_loss: 0.0215 - val_mean_squared_error: 0.0215
Epoch 101/150
1808/1808 [==============================] - 5s 3ms/sample - loss: 0.0154 - mean_squared_error: 0.0154 - val_loss: 0.0211 - val_mean_squared_error: 0.0211
Epoch 102/150
1808/1808 [==============================] - 4s 2ms/sample - loss: 0.0154 - mean_squared_error: 0.0154 - val_loss: 0.0212 - val_mean_squared_error: 0.0212
Epoch 103/150
1808/1808 [==============================] - 4s 2ms/sample - loss: 0.0152 - mean_squared_error: 0.0152 - val_loss: 0.0215 - val_mean_squared_er